In [3]:
import numpy as np
from collections import defaultdict
from load_data import load_process_essays

import logging
import datetime

## Load and Pre-Process Essays

In [19]:
import pickle
from CrossValidation import cross_validation
from BrattEssay import load_bratt_essays
from IterableFP import flatten
from Settings import Settings
from window_based_tagger_config import get_config

CV_FOLDS = 5
DEV_SPLIT = 0.1

settings = Settings()
#data_set = "SkinCancer"
data_set = "CoralBleaching"

root_folder = settings.data_directory + data_set + "/Thesis_Dataset/"
training_folder = root_folder + "Training" + "/"
test_folder = root_folder + "Test" + "/"
training_pickled = settings.data_directory + data_set + "/Thesis_Dataset/training.pl"
models_folder = root_folder + "Models/Bi-LSTM/"

config = get_config(training_folder)

Results Dir: /Users/simon.hughes/Google Drive/Phd/Results/
Data Dir:    /Users/simon.hughes/Google Drive/Phd/Data/
Root Dir:    /Users/simon.hughes/GitHub/NlpResearch/
Public Data: /Users/simon.hughes/GitHub/NlpResearch/Data/PublicDatasets/


In [20]:
%%time
config = get_config(training_folder)
train_essays = load_process_essays(**config)

902 files found
902 essays processed
CPU times: user 47.9 s, sys: 587 ms, total: 48.4 s
Wall time: 50.3 s


In [21]:
%%time
config = get_config(test_folder)
test_essays = load_process_essays(**config)

226 files found
226 essays processed
CPU times: user 14.1 s, sys: 78.8 ms, total: 14.2 s
Wall time: 14.6 s


## Prepare Tags

In [23]:
def get_unique_tokens(essays):
    tag_freq = defaultdict(int)
    unique_words = set()
    num_wds = 0
    for essay in essays:
        for sentence in essay.sentences:
            for word, tags in sentence:
                num_wds += 1
                unique_words.add(word)
    return unique_words, num_wds

train_unq_words,train_wds = get_unique_tokens(train_essays)
test_unq_words,test_wds  = get_unique_tokens(test_essays)
#len(train_unq_words), len(test_unq_words), len(train_unq_words.union(test_unq_words))
train_wds, test_wds, train_wds + test_wds

(137166, 30699, 167865)